In [3]:
import cv2
import matplotlib.pyplot as plt
import importlib
from ipywidgets import interact
import os

In [4]:
import ipynb.fs.full.wspFirefly as wspFirefly
import ipynb.fs.full.wspCS as wspCuckooSearch
import ipynb.fs.full.wspKH as wspKrillHerd
import ipynb.fs.full.wspEHO as wspElephantHerding

import ipynb.fs.full.plotGraph as plotGraph
import ipynb.fs.full.wspMultithreshold as wspMultithreshold

importlib.reload(wspFirefly)
importlib.reload(wspCuckooSearch)
importlib.reload(wspKrillHerd)
importlib.reload(wspElephantHerding)

importlib.reload(plotGraph)
importlib.reload(wspMultithreshold)

<module 'ipynb.fs.full.wspMultithreshold' (/mnt/6884ACCB84AC9CD6/OneDrive - FEI/Aulas/Graduacao/8 - Semestre/CC8421 - TRABALHO FINAL DE CURSO II/image-segmentation/wspMultithreshold.ipynb)>

In [5]:
def readImage(img_path):
    image = cv2.imread(img_path)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return img

In [6]:
bio_algorithm = ['FFA','KH', 'CS', 'ABC', 'EHO']

In [7]:
folder_dir = "images"

all_images = []

for file in os.listdir(folder_dir):
    if file.endswith('.png') or file.endswith('.jpg'):
        all_images.append(file)

In [8]:
@interact(image=all_images, algorithm=bio_algorithm, dimension=(1, 5))
def main(image, algorithm, dimension):
    img = readImage(f"images/{image}")

    plotGraph.plot_image_histogram(img)
    
    hist, best_thresholds, img_thres = wspMultithreshold.wspMultithreshold(img, algorithm, dimension)

    plotGraph.plot_image_histogram_threshold(img_thres, hist, best_thresholds)

interactive(children=(Dropdown(description='image', options=('lenna.png', 'ocranio.jpg', 'piano.jpg', 'zcam.pn…